In [4]:
pip install flask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\akshay\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\akshay\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
@app.route('/process', methods=['POST'])
def process():
    file = request.files['file']
    filepath = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(filepath)

    # Load the file
    if file.filename.endswith('.xlsx'):
        df = pd.read_excel(filepath)
    else:
        df = pd.read_csv(filepath)

    # 👉 PROCESSING LOGIC STARTS HERE
    # Remove timezone info from all datetime columns
    for col in df.select_dtypes(include=['datetimetz']).columns:
        df[col] = df[col].dt.tz_localize(None)

    # Ensure datetime format
    df['Carrier first scan date'] = pd.to_datetime(df['Carrier first scan date'], errors='coerce')
    df['Promised delivery date'] = pd.to_datetime(df['Promised delivery date'], errors='coerce')

    # Function to get first two weekend dates and their names between two dates
    def find_weekends(start_date, end_date):
        weekends = []
        if pd.notna(start_date) and pd.notna(end_date):
            date_range = pd.date_range(start=start_date.normalize(), end=end_date.normalize())
            for date in date_range:
                if date.weekday() in [5, 6]:  # Saturday or Sunday
                    weekends.append((date, date.strftime('%A')))
                if len(weekends) == 2:
                    break
        while len(weekends) < 2:
            weekends.append((pd.NaT, None))
        return pd.Series([weekends[0][0], weekends[0][1], weekends[1][0], weekends[1][1]])

    # Apply the function to each row
    df[['Weekend Date1', 'Weekend Day1', 'Weekend Date2', 'Weekend Day2']] = df.apply(
        lambda row: find_weekends(row['Carrier first scan date'], row['Promised delivery date']),
        axis=1
    )
    # 👉 PROCESSING LOGIC ENDS HERE

    output_filename = f"processed_{file.filename.rsplit('.', 1)[0]}.xlsx"
    output_path = os.path.join(PROCESSED_FOLDER, output_filename)
    df.to_excel(output_path, index=False)

    return render_template('index.html', download_link=f"/download/{output_filename}")


NameError: name 'app' is not defined